<font size="5"><center> <b>Sandpyper: sandy beaches SfM-UAV analysis tools</b></center></font>
<font size="4"><center> <b> Example 1 - Profiles extraction </b></center> <br>

    
<center><img src="images/banner.png" width="80%"  /></center>

<font face="Calibri">
<br>
<font size="5"> <b>Profiles creation and data extraction from DSM and orthophotos</b></font>

<br>
<font size="4"> <b> Nicolas Pucino; PhD Student @ Deakin University, Australia </b> <br>

<font size="3">The first steps in a typical workflow is to create cross-shore transects in all the locations and extract elevation and RGB information along those transects. Sandpiper allows the data extraction from hundreds of rasters at once, in an organised way. <br>

<b>This notebook covers the following concepts:</b>

- Naming conventions and global parameters.
- Setting up the folders.
- Setting up the folders.
</font>


</font>

Import all it is required.

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import os

from sandpyper.outils import cross_ref,create_spatial_id
from sandpyper.dynamics import compute_multitemporal
from sandpyper.profile import extract_from_folder
from sandpyper.space import create_transects

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
class ProfileSet():
    """
    ciao
    """
    def __init__(self,
                 dirNameDSM,
                 dirNameOrtho,
                 dirNameTrans,
                 loc_codes,
                 loc_search_dict,
                 crs_dict_string,
                check='all'):
        
        
        self.dirNameDSM=dirNameDSM
        self.dirNameOrtho=dirNameOrtho
        self.dirNameTrans=dirNameTrans
        
        self.loc_codes=loc_codes
        self.loc_search_dict=loc_search_dict
        self.crs_dict_string=crs_dict_string
        
        if check=="dsm":
            path_in=self.dirNameDSM
        elif check == "ortho":
            path_in=self.dirNameOrtho
        elif check == "all":
            path_in=[self.dirNameDSM, self.dirNameOrtho]
            
        
        self.check=cross_ref(path_in,
                        self.dirNameTrans,
                        print_info=True, 
                        loc_search_dict=self.loc_search_dict,
                        list_loc_codes=self.loc_codes)

        
    def extract_profiles(self,
                         mode,
                         sampling_step,
                         add_xy,
                         add_slope=False,
                         default_nan_values=-10000):
        
        if mode=="dsm":
            path_in=self.dirNameDSM
        elif mode == "ortho":
            path_in=self.dirNameOrtho
        elif mode == "all":
            path_in=[self.dirNameDSM,self.dirNameOrtho]
        else:
            raise NameError("mode must be either 'dsm','ortho' or 'all'.")
        
        if mode in ["dsm","ortho"]:
            
            profiles=extract_from_folder(dataset_folder=path_in,
                transect_folder=self.dirNameTrans,
                mode=mode,sampling_step=sampling_step,
                list_loc_codes=self.loc_codes,
                add_xy=add_xy,
                add_slope=add_slope,
                default_nan_values=default_nan_values)
            
            profiles["distance"]=np.round(profiles.loc[:,"distance"].values.astype("float"),2)
            
        elif mode == "all":
            
            print("Extracting elevation from DSMs . . .")
            profiles_z=extract_from_folder( dataset_folder=path_in[0],
                    transect_folder=self.dirNameTrans,
                    mode="dsm",
                    sampling_step=sampling_step,
                    list_loc_codes=self.loc_codes,
                    add_xy=add_xy,
                    add_slope=add_slope,
                    default_nan_values=default_nan_values )
                        
            print("Extracting rgb values from orthos . . .")
            profiles_rgb=extract_from_folder(dataset_folder=path_in[1],
                transect_folder=self.dirNameTrans,
                mode="ortho",sampling_step=sampling_step,
                list_loc_codes=self.loc_codes,
                add_xy=add_xy,
                default_nan_values=default_nan_values)
            
            profiles_rgb["distance"]=np.round(profiles_rgb.loc[:,"distance"].values.astype("float"),2)
            profiles_z["distance"]=np.round(profiles_z.loc[:,"distance"].values.astype("float"),2)

            profiles_merged = pd.merge(profiles_z,profiles_rgb[["band1","band2","band3","point_id"]],on="point_id",validate="one_to_one")
            profiles_merged=profiles_merged.replace("", np.NaN)
            profiles_merged['z']=profiles_merged.z.astype("float")
            
            self.profiles=profiles_merged
            
        else:
            raise NameError("mode must be either 'dsm','ortho' or 'all'.")
        
        self.sampling_step=sampling_step
        
    def compute_multitemporal(self, date_field='raw_date',
                              filter_sand=False,
                             sand_label_field='label_sand'):
        
        dh_df=compute_multitemporal(self.profiles,
                     date_field=date_field, filter_sand=filter_sand,
                     sand_label_field=sand_label_field)
            


In [3]:
dirNameDSM=r'C:\my_packages\sandpyper\tests\test_data\dsm_1m'

dirNameOrtho=r'C:\my_packages\sandpyper\tests\test_data\orthos_1m'

dirNameTrans=r'C:\my_packages\sandpyper\tests\test_data\transects'

loc_codes=["mar","leo"]
loc_search_dict = {   'leo': ['St','Leonards','leonards','leo'],
                      'mar': ['Marengo','marengo','mar'] }
crs_dict_string= {
                 'mar': {'init': 'epsg:32754'},
                 'leo':{'init': 'epsg:32755'}
                 }


In [4]:
P=ProfileSet(dirNameDSM,
            dirNameOrtho,
            dirNameTrans,
            loc_codes,
            loc_search_dict,
            crs_dict_string,
            check="all")

C:\conda3\envs\sandpyper_env\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


dsm from leo = 6

ortho from leo = 6

dsm from mar = 9

ortho from mar = 9


NUMBER OF DATASETS TO PROCESS: 30


In [5]:
P.extract_profiles('all',1,True)

Extracting elevation from DSMs . . .


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

...

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

Extraction succesfull
Number of points extracted:32805
Time for processing=61.61892795562744 seconds
First 10 rows are printed below
Number of points outside the raster extents: 27198
The extraction assigns NaN.
Number of points in NoData areas within the raster extents: 0
The extraction assigns NaN.


In [13]:
df=P.profiles
date_field="raw_date"
filter_sand=False
sand_label_field='label_sand'
from tqdm.notebook import tqdm


In [30]:
for i,col in enumerate(merged.columns):
    print(i,col)

0 distance_pre
1 z_pre
2 tr_id_pre
3 raw_date_pre
4 coordinates_pre
5 location_pre
6 survey_date_pre
7 point_id_pre
8 x_pre
9 y_pre
10 band1_pre
11 band2_pre
12 band3_pre
13 spatial_id
14 distance_post
15 z_post
16 tr_id_post
17 raw_date_post
18 coordinates_post
19 location_post
20 survey_date_post
21 point_id_post
22 x_post
23 y_post
24 band1_post
25 band2_post
26 band3_post
27 dh


In [38]:

merged.filter(like=geometry_column).iloc[:,0]

0       POINT (731646.904 5705523.469)
1       POINT (731646.078 5705524.033)
2       POINT (731645.253 5705524.598)
3       POINT (731644.427 5705525.162)
4       POINT (731643.602 5705525.727)
                     ...              
1674    POINT (731437.893 5705159.623)
1675    POINT (731436.899 5705159.730)
1676    POINT (731435.905 5705159.838)
1677    POINT (731434.911 5705159.945)
1678    POINT (731433.916 5705160.052)
Name: coordinates_pre, Length: 1679, dtype: geometry

In [14]:
df["spatial_id"]=[create_spatial_id(df.iloc[i]) for i in range(df.shape[0])]
fusion_long=pd.DataFrame()
geometry_column="coordinates"

for location in df.location.unique():
    print(f"working on {location}")
    loc_data=df.query(f"location=='{location}'")
    list_dates=loc_data.loc[:,date_field].unique()
    list_dates.sort()


    for i in tqdm(range(list_dates.shape[0])):

        if i < list_dates.shape[0]-1:
            date_pre=list_dates[i]
            date_post=list_dates[i+1]
            print(f"Calculating dt{i}, from {date_pre} to {date_post} in {location}.")

            if filter_sand:
                df_pre=loc_data.query(f"{date_field} =='{date_pre}' & {sand_label_field} in {filter_classes}").dropna(subset=['z'])
                df_post=loc_data.query(f"{date_field} =='{date_post}' & {sand_label_field} in {filter_classes}").dropna(subset=['z'])
            else:
                df_pre=loc_data.query(f"{date_field} =='{date_pre}'").dropna(subset=['z'])
                df_post=loc_data.query(f"{date_field} =='{date_post}'").dropna(subset=['z'])

            merged=pd.merge(df_pre,df_post, how='inner', on='spatial_id', validate="one_to_one",suffixes=('_pre','_post'))
            merged["dh"]=merged.z_post.astype(float) - merged.z_pre.astype(float)

            dict_short={"geometry":merged.filter(like=geometry_column).iloc[:,0],
                        "location":location,
                        "tr_id":merged.tr_id_pre,
                        "distance":merged.distance_pre,
                        "dt":  f"dt_{i}",
                        "date_pre":date_pre,
                        "date_post":date_post,
                        "z_pre":merged.z_pre.astype(float),
                        "z_post":merged.z_post.astype(float),
                        "dh":merged.dh}

            short_df=pd.DataFrame(dict_short)
            fusion_long=pd.concat([short_df,fusion_long],ignore_index=True)

working on mar


  0%|          | 0/9 [00:00<?, ?it/s]

Calculating dt0, from 20180601 to 20180621 in mar.


AttributeError: 'GeoDataFrame' object has no attribute 'geometry_pre'

In [16]:
df

,distance,z,tr_id,raw_date,coordinates,location,survey_date,point_id,x,y,band1,band2,band3,spatial_id
0,0.0,0.007440,21,20190516,POINT (731646.904 5705523.469),mar,2019-05-16,61121091m2580400ar00,731646.903760,5.705523e+06,114.0,139.0,128.0,0ma2r0100
1,1.0,0.008439,21,20190516,POINT (731646.078 5705524.033),mar,2019-05-16,61123091m2580600ar10,731646.078301,5.705524e+06,117.0,139.0,127.0,0ma2r0110
2,2.0,0.010800,21,20190516,POINT (731645.253 5705524.598),mar,2019-05-16,61129091m2530100ar20,731645.252842,5.705525e+06,122.0,140.0,127.0,0ma2r0120
3,3.0,0.011350,21,20190516,POINT (731644.427 5705525.162),mar,2019-05-16,61124091m2570800ar30,731644.427383,5.705525e+06,125.0,144.0,133.0,0ma2r0130
4,4.0,0.028030,21,20190516,POINT (731643.602 5705525.727),mar,2019-05-16,61120091m2520400ar40,731643.601924,5.705526e+06,126.0,145.0,133.0,0ma2r0140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32800,44.0,NaN,0,20180606,POINT (300201.502 5772859.891),leo,2018-06-06,60101080l2624100eo40,300201.501848,5.772860e+06,NaN,NaN,NaN,0le0o0044
32801,45.0,NaN,0,20180606,POINT (300202.298 5772860.496),leo,2018-06-06,60100080l2665200eo40,300202.298010,5.772860e+06,NaN,NaN,NaN,0le0o0045
32802,46.0,NaN,0,20180606,POINT (300203.094 5772861.101),leo,2018-06-06,60100080l2606300eo40,300203.094172,5.772861e+06,NaN,NaN,NaN,0le0o0046
32803,47.0,NaN,0,20180606,POINT (300203.890 5772861.706),leo,2018-06-06,60109080l2647400eo40,300203.890334,5.772862e+06,NaN,NaN,NaN,0le0o0047


In [15]:
merged

,distance_pre,z_pre,tr_id_pre,raw_date_pre,coordinates_pre,location_pre,survey_date_pre,point_id_pre,x_pre,y_pre,...,coordinates_post,location_post,survey_date_post,point_id_post,x_post,y_post,band1_post,band2_post,band3_post,dh
0,0.0,0.792593,21,20180601,POINT (731646.904 5705523.469),mar,2018-06-01,11121080m2680400ar00,731646.903760,5.705523e+06,...,POINT (731646.904 5705523.469),mar,2018-06-21,11121082m2680400ar00,731646.903760,5.705523e+06,117.0,154.0,153.0,-0.625075
1,1.0,0.781489,21,20180601,POINT (731646.078 5705524.033),mar,2018-06-01,11123080m2680600ar10,731646.078301,5.705524e+06,...,POINT (731646.078 5705524.033),mar,2018-06-21,11123082m2680600ar10,731646.078301,5.705524e+06,120.0,158.0,157.0,-0.613956
2,2.0,0.789588,21,20180601,POINT (731645.253 5705524.598),mar,2018-06-01,11129080m2630100ar20,731645.252842,5.705525e+06,...,POINT (731645.253 5705524.598),mar,2018-06-21,11129082m2630100ar20,731645.252842,5.705525e+06,110.0,147.0,143.0,-0.622113
3,3.0,0.776424,21,20180601,POINT (731644.427 5705525.162),mar,2018-06-01,11124080m2670800ar30,731644.427383,5.705525e+06,...,POINT (731644.427 5705525.162),mar,2018-06-21,11124082m2670800ar30,731644.427383,5.705525e+06,110.0,149.0,143.0,-0.609002
4,4.0,0.794274,21,20180601,POINT (731643.602 5705525.727),mar,2018-06-01,11120080m2620400ar40,731643.601924,5.705526e+06,...,POINT (731643.602 5705525.727),mar,2018-06-21,11120082m2620400ar40,731643.601924,5.705526e+06,113.0,151.0,146.0,-0.626869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,75.0,4.686693,0,20180601,POINT (731437.893 5705159.623),mar,2018-06-01,10102080m2635900ar70,731437.893301,5.705160e+06,...,POINT (731437.893 5705159.623),mar,2018-06-21,10102082m2635900ar70,731437.893301,5.705160e+06,79.0,96.0,90.0,7.601466
1675,76.0,4.493311,0,20180601,POINT (731436.899 5705159.730),mar,2018-06-01,10109080m2666800ar70,731436.899062,5.705160e+06,...,POINT (731436.899 5705159.730),mar,2018-06-21,10109082m2666800ar70,731436.899062,5.705160e+06,60.0,73.0,64.0,6.050527
1676,77.0,5.125780,0,20180601,POINT (731435.905 5705159.838),mar,2018-06-01,10106080m2697800ar70,731435.904823,5.705160e+06,...,POINT (731435.905 5705159.838),mar,2018-06-21,10106082m2697800ar70,731435.904823,5.705160e+06,45.0,57.0,53.0,2.428983
1677,78.0,8.266596,0,20180601,POINT (731434.911 5705159.945),mar,2018-06-01,10104080m2628800ar70,731434.910584,5.705160e+06,...,POINT (731434.911 5705159.945),mar,2018-06-21,10104082m2628800ar70,731434.910584,5.705160e+06,51.0,63.0,54.0,-0.840238


In [6]:
P.compute_multitemporal()

working on mar


  0%|          | 0/9 [00:00<?, ?it/s]

Calculating dt0, from 20180601 to 20180621 in mar.


AttributeError: 'GeoDataFrame' object has no attribute 'geometry_pre'

In [ ]:
P.profiles_z.query("location=='mar'").plot(column="z")

## Global parameters
### Location codes
When your analysis involves a multi-site approach, it is convenient to assign each location a __small code__ to easy the handling of every associated file and its coordinate reference system.

Here are some examples:
* Saint Leonards : __leo__
* Marengo : __mar__

In [ ]:
# The location codes used troughout the analysis
loc_codes=["mar","leo"]

### Location search dictionary
Sometimes, we need to automatically obtain the right location code from raster files, either of Digital Surface Models (DSM) or Orthophotos (ORTO), whose filenames contains the original location name (e.g. Saint_Leonards or Warrnambool).

An easy and fast way to do this, is to create a dictionary, where __keys are the location codes__ and the __values are lists of possible full names__ we expect to find in the files.
Here are some examples:

```python
loc_search_dict = {   'leo': ['St','Leonards','leonards','leo'],
                      'mar': ['Marengo','marengo','mar'] }
```
> __NOTE__: always include the location codes in the list of possible names, in case the original raster filenames are already formatted!


In [ ]:
# The terms used in the original filenames.
# These will be used to properly format files, extracting location codes and dates.

loc_search_dict = {   'leo': ['St','Leonards','leonards','leo'],
                      'mar': ['Marengo','marengo','mar'] }

### Coordinate Reference Systems dictionary

Working on a wide area, often requires dealing with multiple Coordinate Reference Systems (CRS). Therefore, it is important to assign __each location code with its appropriate CRS__ at the beginning, in order to always take it into account trhoughout the analysis.

We do this with another dictionary, called `crs_dict_string`, where as keys we store the location codes and as values we store another dictionary, in this form `{'init': 'epsg:32754'}` .
Modify the __EPSG code__ to change CRS. Here is an example of the resulting dictionary:

```python
crs_dict_string = {'wbl': {'init': 'epsg:32754'},
                   'apo': {'init': 'epsg:32754'},
                   'prd': {'init': 'epsg:32755'},
                   'dem': {'init': 'epsg:32755'} }
```



> __NOTE:__ to specify the CRS, use the dictionary format supported by Geopandas 0.6.3. Only projected CRS are supported.


In [ ]:
# The Coordinate Reference Systems used troughout this example

crs_dict_string= {
                 'mar': {'init': 'epsg:32754'},
                 'leo':{'init': 'epsg:32755'}
                 }

## Transects creation
### Shoreline baseline

Any Shapely line or polyline object can be used as input as a transect. In coastal geomeorphometric studies, transects are usually equally spaced alongshore, and place normal to the shoreline. However, any type of line can be used to extract values from both orthophtos and DSMs.

You can construct transects in 2 ways:
1. **Any GIS**
2. Using the function **create_transects**, starting from a __shoreline baseline__.

If you use your favourite GIS (Qgis preferred), ensure that the output format is __geopackage (.gpkg)__, and:
> **each transect must be in a separate row (geometry)**

If you want to use in-built sandpyper function, see below:

In [ ]:
# load and display the shoreline
path_to_shoreline_mar=r'C:\my_packages\doc_data\test_data\shorelines\leo_shoreline_short.gpkg' # Marengo shoreline

shoreline=gpd.read_file(path_to_shoreline_mar)
shoreline.plot()

In [ ]:
# create and display the transects (in red)

f,ax=plt.subplots(figsize=(10,10))  # Change figsize if you want bigger images
location='mar'  # insert the location code for this transect.

transects=create_transects(shoreline,
                           sampling_step=20, # alongshore spacing
                           tick_length=50, # transects length
                           location=location,crs=crs_dict_string[location],
                           side='both' # 'both':transect is centered at the interesction with the baseline
                          )

# Modify the figure by plotting shoreline, transects and transect IDs.
shoreline.plot(ax=ax,color='b')
transects.plot(ax=ax,color='r')

for x, y, label in zip(transects.geometry.centroid.x, transects.geometry.centroid.y, transects.tr_id):
    ax.annotate(label, xy=(x, y), xytext=(4, 1), textcoords="offset points")

transects.head()

The following cell saves transect to file and name it as:

>__locationCode_whateverYouWant.gpkg__

(example: __leo_transects.gpkg__)

and place it to a folder where you store all the transects for each location.

>__Note:__ If the saving throws an error "PLE_NotSupported in dataset leo_transects.gpkg does not support layer creation option ENCODING", the file should be created and valid anyway. Double-check by opening it in Qgis.

In [ ]:
transects.to_file(filename=r'C:\my_packages\doc_data\transects\leo_transects.gpkg',driver='GPKG')

## Elevation and RGB data extraction

### Define the folders containing the datasets

First, let's define the paths to the folders containing the raster DSMs or ORTOs and the transects.

> __SUPPORTED FORMATS:__
>* Rasters: __geotiffs (.tif, .tiff)__
>* Transects: __geopackages (.gpkg).__
>
>__Transects filenames:__ when creating transects, save them with the location code as filename.

<img src="images/hill_ortho_profiles_cow.png" width="85%" />

In [ ]:
# Set the path to the folders containing the DSMs (dirNameDSM) and the transect files (dirNameTrans)

dirNameDSM=r'C:\my_packages\sandpyper\tests\test_data\dsm_1m'

dirNameOrtho=r'C:\my_packages\sandpyper\tests\test_data\orthos_1m'

dirNameTrans=r'C:\my_packages\sandpyper\tests\test_data\transects'


###  Check cross-reference table and CRS matches

The EPSG codes in the raster and transect columns __must match__.

Please also check the dsm and date columns match transect file and dsm file paths.

In [ ]:
check=cross_ref(dirNameDSM,dirNameTrans,print_info=True, loc_search_dict=loc_search_dict,list_loc_codes=loc_codes)
check

Nice! We are now ready to extract data from DSMs or ORTOs with the provided transect files.

### Extraction of profiles from folder

This is the cell where the automatic extraction gets processed.
The only parameter to set is the __sampling step__ variable, which indicates the __cross-shore sampling distance (m)__ that we want to use along our transects. Beware, although you could use a very small sampling distance (UAV datasets tend to be between few to 10 cm pixel size), file dimension will increase significantly!.

__Dealing with NaNs__

>NaNs might come from two different cases:
>1. extraction of points generated on transects falling __outside__ of the underlying raster extent
>2. points sampled from transect __inside__ the raster extent but containing NoData cells.
>
>Conveniently, the extraction profile function makes sure that if points fall outside the raster extent (case 1), those >elevations are assigned a default nan value, in the NumPy np.nan form.
>In case 2, however, the values extracted depends on the definition of NaNs of the source raster format.

In [ ]:
%%time

## Parameters to specify

transect_folder=dirNameTrans
sampling_step=1


gdf_rgb=extract_from_folder(dataset_folder=dirNameOrtho,
                        transect_folder=transect_folder,
                        mode="ortho",sampling_step=sampling_step,
                        list_loc_codes=loc_codes,
                        add_xy=True)

gdf=extract_from_folder(dataset_folder=dirNameDSM,
                        transect_folder=transect_folder,
                        mode="dsm",sampling_step=sampling_step,
                        list_loc_codes=loc_codes,
                        add_xy=True)


In [ ]:
gdf.head()

### GOOD!

save the Geodataframes (gdf and gdf_rgb) as a CSV file and head to the __SANDPYPER Labeling sand notebook__.

In [ ]:
# Saving the files

gdf.to_csv(r"C:\my_packages\sandpyper\tests\test_outputs\gdf.csv",index=False)
gdf_rgb.to_csv(r"C:\my_packages\sandpyper\tests\test_outputs\gdf_rgb.csv",index=False)

___